# GraphRAG Implementation with LlamaIndex - V2

https://docs.llamaindex.ai/en/stable/examples/cookbooks/GraphRAG_v2/

## 失敗：build_communities() 被 Azuer OpenAI 暴力內容被擋下來

BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy.

## Config

In [1]:
import os

from dotenv import load_dotenv
load_dotenv(".env")

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

# import logging
# import sys

# logging.basicConfig(
#     stream=sys.stdout, level=logging.INFO
# )  # logging.DEBUG for more verbose output
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

gpt4o = AzureOpenAI(
    model="gpt-4o",
    deployment_name=os.environ["MY_AZURE_OPENAI_DEPLOYMENT_NAME_CHAT"], 
    api_key=os.environ["MY_AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["MY_AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-3-small",
    deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME_EMBEDDINGS"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

from llama_index.core import Settings

Settings.llm = gpt4o
Settings.embed_model = embed_model

## Data

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./data/90-Romance-of-the-Three-Kingdoms.txt", "./data/91-Romance-of-the-Three-Kingdoms.txt", "./data/92-Romance-of-the-Three-Kingdoms.txt"]
).load_data()

documents

[Document(id_='fa2059b0-a8e9-4c38-9311-e50201f58886', embedding=None, metadata={'file_path': 'data/90-Romance-of-the-Three-Kingdoms.txt', 'file_name': '90-Romance-of-the-Three-Kingdoms.txt', 'file_type': 'text/plain', 'file_size': 20175, 'creation_date': '2025-01-16', 'last_modified_date': '2025-01-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='第九十回：驅巨獸六破蠻兵，燒藤甲七擒孟獲\n卻說孔明放了孟獲等一干人，楊鋒父子皆封官爵，重賞洞兵。楊鋒等拜謝而去。孟獲等連夜奔回銀坑洞。那洞外有三江：乃是瀘水、甘南水、西城水。三路水會合，故為三江。其洞北近平坦二百餘里，多產萬物；洞西二百餘里，有鹽井；西南二百里，直抵瀘、甘；正南三百里，乃是梁都洞。洞中有山，環抱其洞；山上出銀礦，故名為銀坑山。山中置宮殿樓臺，以為蠻王巢穴。\n其中建一祖廟，名曰「家鬼」。四時殺牛宰馬享祭。名曰「卜鬼」。每年常以蜀人并外鄉之人祭之。若人患病，不肯服藥，只禱師巫，名為「藥鬼。」其處無刑法，但犯罪即斬。有女長成，卻於溪中沐浴，男女自相混淆，任其自配，父

## GraphRAGExtractor

In [3]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

from typing import Any, List, Callable, Optional, Union, Dict
from IPython.display import Markdown, display

from llama_index.core.async_utils import run_jobs
from llama_index.core.indices.property_graph.utils import (
    default_parse_triplets_fn,
)
from llama_index.core.graph_stores.types import (
    EntityNode,
    KG_NODES_KEY,
    KG_RELATIONS_KEY,
    Relation,
)
from llama_index.core.llms.llm import LLM
from llama_index.core.prompts import PromptTemplate
from llama_index.core.prompts.default_prompts import (
    DEFAULT_KG_TRIPLET_EXTRACT_PROMPT,
)
from llama_index.core.schema import TransformComponent, BaseNode
from llama_index.core.bridge.pydantic import BaseModel, Field


class GraphRAGExtractor(TransformComponent):
    """Extract triples from a graph.

    Uses an LLM and a simple prompt + output parsing to extract paths (i.e. triples) and entity, relation descriptions from text.

    Args:
        llm (LLM):
            The language model to use.
        extract_prompt (Union[str, PromptTemplate]):
            The prompt to use for extracting triples.
        parse_fn (callable):
            A function to parse the output of the language model.
        num_workers (int):
            The number of workers to use for parallel processing.
        max_paths_per_chunk (int):
            The maximum number of paths to extract per chunk.
    """

    llm: LLM
    extract_prompt: PromptTemplate
    parse_fn: Callable
    num_workers: int
    max_paths_per_chunk: int

    def __init__(
        self,
        llm: Optional[LLM] = None,
        extract_prompt: Optional[Union[str, PromptTemplate]] = None,
        parse_fn: Callable = default_parse_triplets_fn,
        max_paths_per_chunk: int = 10,
        num_workers: int = 4,
    ) -> None:
        """Init params."""
        from llama_index.core import Settings

        if isinstance(extract_prompt, str):
            extract_prompt = PromptTemplate(extract_prompt)
            # extract_prompt = (
            #     "Some text is provided below. Given the text, extract up to "
            #     "{max_knowledge_triplets} "
            #     "knowledge triplets in the form of (subject, predicate, object). Avoid stopwords and use Traditional Chinese.\n"
            #     "---------------------\n"
            #     "Example:"
            #     "Text: 小美是小明的媽媽"
            #     "Triplets:\n(小美, 是母親, 小明)\n"
            #     "Text: 曹操在官渡之戰中擊敗了袁紹。"
            #     "Triplets:"
            #     "(曹操, 參與, 官渡之戰)"
            #     "(曹操, 擊敗了, 袁紹)"

            #     "---------------------\n"
            #     "Text: {text}\n"
            #     "Triplets:\n"
            # )

        super().__init__(
            llm=llm or Settings.llm,
            extract_prompt=extract_prompt or DEFAULT_KG_TRIPLET_EXTRACT_PROMPT,
            parse_fn=parse_fn,
            num_workers=num_workers,
            max_paths_per_chunk=max_paths_per_chunk,
        )

    @classmethod
    def class_name(cls) -> str:
        return "GraphExtractor"

    def __call__(
        self, nodes: List[BaseNode], show_progress: bool = False, **kwargs: Any
    ) -> List[BaseNode]:
        """Extract triples from nodes."""
        return asyncio.run(
            self.acall(nodes, show_progress=show_progress, **kwargs)
        )

    async def _aextract(self, node: BaseNode) -> BaseNode:
        """Extract triples from a node."""
        assert hasattr(node, "text")

        text = node.get_content(metadata_mode="llm")
        try:
            llm_response = await self.llm.apredict(
                self.extract_prompt,
                text=text,
                max_knowledge_triplets=self.max_paths_per_chunk,
            )
            entities, entities_relationship = self.parse_fn(llm_response)
        except ValueError:
            entities = []
            entities_relationship = []

        existing_nodes = node.metadata.pop(KG_NODES_KEY, [])
        existing_relations = node.metadata.pop(KG_RELATIONS_KEY, [])
        entity_metadata = node.metadata.copy()
        for entity, entity_type, description in entities:
            entity_metadata["entity_description"] = description
            entity_node = EntityNode(
                name=entity, label=entity_type, properties=entity_metadata
            )
            existing_nodes.append(entity_node)

        relation_metadata = node.metadata.copy()
        for triple in entities_relationship:
            subj, obj, rel, description = triple
            relation_metadata["relationship_description"] = description
            rel_node = Relation(
                label=rel,
                source_id=subj,
                target_id=obj,
                properties=relation_metadata,
            )

            existing_relations.append(rel_node)

        node.metadata[KG_NODES_KEY] = existing_nodes
        node.metadata[KG_RELATIONS_KEY] = existing_relations
        return node

    async def acall(
        self, nodes: List[BaseNode], show_progress: bool = False, **kwargs: Any
    ) -> List[BaseNode]:
        """Extract triples from nodes async."""
        jobs = []
        for node in nodes:
            jobs.append(self._aextract(node))

        return await run_jobs(
            jobs,
            workers=self.num_workers,
            show_progress=show_progress,
            desc="Extracting paths from text",
        )

## GraphRAGStore

In [4]:
import re
import networkx as nx
from graspologic.partition import hierarchical_leiden
from collections import defaultdict

from llama_index.core.llms import ChatMessage
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore


class GraphRAGStore(Neo4jPropertyGraphStore):
    community_summary = {}
    entity_info = None
    max_cluster_size = 5

    def __init__(self, username, password, url):
        super().__init__(username=username, password=password, url=url)

    def generate_community_summary(self, text):
        """Generate summary for a given text using an LLM."""
        messages = [
            ChatMessage(
                role="system",
                content=(
                    "You are provided with a set of relationships from a knowledge graph, each represented as "
                    "entity1->entity2->relation->relationship_description. Your task is to create a summary of these "
                    "relationships. The summary should include the names of the entities involved and a concise synthesis "
                    "of the relationship descriptions. The goal is to capture the most critical and relevant details that "
                    "highlight the nature and significance of each relationship. Ensure that the summary is coherent and "
                    "integrates the information in a way that emphasizes the key aspects of the relationships.You must using Traditional Chinese."
                ),
            ),
            ChatMessage(role="user", content=text),
        ]
        response = gpt4o.chat(messages)
        clean_response = re.sub(r"^assistant:\s*", "", str(response)).strip()
        return clean_response

    def build_communities(self):
        """Builds communities from the graph and summarizes them."""
        nx_graph = self._create_nx_graph()
        community_hierarchical_clusters = hierarchical_leiden(
            nx_graph, max_cluster_size=self.max_cluster_size
        )
        self.entity_info, community_info = self._collect_community_info(
            nx_graph, community_hierarchical_clusters
        )
        self._summarize_communities(community_info)

    def _create_nx_graph(self):
        """Converts internal graph representation to NetworkX graph."""
        nx_graph = nx.Graph()
        triplets = self.get_triplets()
        for entity1, relation, entity2 in triplets:
            nx_graph.add_node(entity1.name)
            nx_graph.add_node(entity2.name)
            nx_graph.add_edge(
                relation.source_id,
                relation.target_id,
                relationship=relation.label,
                description=relation.properties["relationship_description"],
            )
        return nx_graph

    def _collect_community_info(self, nx_graph, clusters):
        """
        Collect information for each node based on their community,
        allowing entities to belong to multiple clusters.
        """
        entity_info = defaultdict(set)
        community_info = defaultdict(list)

        for item in clusters:
            node = item.node
            cluster_id = item.cluster

            # Update entity_info
            entity_info[node].add(cluster_id)

            for neighbor in nx_graph.neighbors(node):
                edge_data = nx_graph.get_edge_data(node, neighbor)
                if edge_data:
                    detail = f"{node} -> {neighbor} -> {edge_data['relationship']} -> {edge_data['description']}"
                    community_info[cluster_id].append(detail)

        # Convert sets to lists for easier serialization if needed
        entity_info = {k: list(v) for k, v in entity_info.items()}

        return dict(entity_info), dict(community_info)

    def _summarize_communities(self, community_info):
        """Generate and store summaries for each community."""
        for community_id, details in community_info.items():
            details_text = (
                "\n".join(details) + "."
            )  # Ensure it ends with a period
            self.community_summary[
                community_id
            ] = self.generate_community_summary(details_text)

    def get_community_summaries(self):
        """Returns the community summaries, building them if not already done."""
        if not self.community_summary:
            self.build_communities()
        return self.community_summary

## GraphRAGQueryEngine

In [5]:
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.llms import LLM
from llama_index.core import PropertyGraphIndex

import re


class GraphRAGQueryEngine(CustomQueryEngine):
    graph_store: GraphRAGStore
    index: PropertyGraphIndex
    llm: LLM
    similarity_top_k: int = 20

    def custom_query(self, query_str: str) -> str:
        """Process all community summaries to generate answers to a specific query."""

        entities = self.get_entities(query_str, self.similarity_top_k)

        community_ids = self.retrieve_entity_communities(
            self.graph_store.entity_info, entities
        )
        community_summaries = self.graph_store.get_community_summaries()
        community_answers = [
            self.generate_answer_from_summary(community_summary, query_str)
            for id, community_summary in community_summaries.items()
            if id in community_ids
        ]

        final_answer = self.aggregate_answers(community_answers)
        return final_answer

    def get_entities(self, query_str, similarity_top_k):
        nodes_retrieved = self.index.as_retriever(
            similarity_top_k=similarity_top_k
        ).retrieve(query_str)

        enitites = set()
        pattern = (
            r"^(\w+(?:\s+\w+)*)\s*->\s*([a-zA-Z\s]+?)\s*->\s*(\w+(?:\s+\w+)*)$"
        )

        for node in nodes_retrieved:
            matches = re.findall(
                pattern, node.text, re.MULTILINE | re.IGNORECASE
            )

            for match in matches:
                subject = match[0]
                obj = match[2]
                enitites.add(subject)
                enitites.add(obj)

        return list(enitites)

    def retrieve_entity_communities(self, entity_info, entities):
        """
        Retrieve cluster information for given entities, allowing for multiple clusters per entity.

        Args:
        entity_info (dict): Dictionary mapping entities to their cluster IDs (list).
        entities (list): List of entity names to retrieve information for.

        Returns:
        List of community or cluster IDs to which an entity belongs.
        """
        community_ids = []

        for entity in entities:
            if entity in entity_info:
                community_ids.extend(entity_info[entity])

        return list(set(community_ids))

    def generate_answer_from_summary(self, community_summary, query):
        """Generate an answer from a community summary based on a given query using LLM."""
        prompt = (
            f"Given the community summary: {community_summary}, "
            f"how would you answer the following query? Answer in Traditional Chinese. Query: {query}"
        )
        messages = [
            ChatMessage(role="system", content=prompt),
            ChatMessage(
                role="user",
                content="I need an answer based on the above information.",
            ),
        ]
        response = self.llm.chat(messages)
        cleaned_response = re.sub(r"^assistant:\s*", "", str(response)).strip()
        return cleaned_response

    def aggregate_answers(self, community_answers):
        """Aggregate individual community answers into a final, coherent response."""
        # intermediate_text = " ".join(community_answers)
        prompt = "Combine the following intermediate answers into a final, concise response and answer in Traditional Chinese."
        messages = [
            ChatMessage(role="system", content=prompt),
            ChatMessage(
                role="user",
                content=f"Intermediate answers: {community_answers}",
            ),
        ]
        final_response = self.llm.chat(messages)
        cleaned_final_response = re.sub(
            r"^assistant:\s*", "", str(final_response)
        ).strip()
        return cleaned_final_response

## Build End to End GraphRAG Pipeline

In [6]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=250,
    chunk_overlap=24,
)
nodes = splitter.get_nodes_from_documents(documents)

len(nodes)

134

### Build ProperGraphIndex using GraphRAGExtractor and GraphRAGStore

In [7]:
KG_TRIPLET_EXTRACT_TMPL = """
-Goal-
Given a text document, identify all entities and their entity types from the text and all relationships among the identified entities.
Given the text, extract up to {max_knowledge_triplets} entity-relation triplets.

-Steps-
1. Identify all entities. For each identified entity, extract the following information:
- entity_name: Name of the entity, use Traditional Chinese only.
- entity_type: Type of the entity
- entity_description: Comprehensive description of the entity's attributes and activities, use Traditional Chinese only.
Format each entity as ("entity"$$$$""$$$$""$$$$"")

2. From the entities identified in step 1, identify all pairs of (source_entity, target_entity) that are *clearly related* to each other.
For each pair of related entities, extract the following information:
- source_entity: name of the source entity, as identified in step 1
- target_entity: name of the target entity, as identified in step 1
- relation: relationship between source_entity and target_entity
- relationship_description: explanation as to why you think the source entity and the target entity are related to each other, use Traditional Chinese only.

Format each relationship as ("relationship"$$$$""$$$$""$$$$""$$$$"")

3. When finished, output.

-Real Data-
######################
text: {text}
######################
output:"""

In [8]:
entity_pattern = r'\("entity"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\)'
relationship_pattern = r'\("relationship"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\)'


def parse_fn(response_str: str) -> Any:
    entities = re.findall(entity_pattern, response_str)
    relationships = re.findall(relationship_pattern, response_str)
    return entities, relationships


kg_extractor = GraphRAGExtractor(
    llm=gpt4o,
    extract_prompt=KG_TRIPLET_EXTRACT_TMPL,
    max_paths_per_chunk=2,
    parse_fn=parse_fn,
)

### Neo4j

In [9]:
from llama_index.core import PropertyGraphIndex

graph_store = GraphRAGStore(
    username=os.environ["LI_NEO4J_USERNAME"],
    password=os.environ["LI_NEO4J_PASSWORD"],
    url=os.environ["LI_NEO4J_URI"],
)

index = PropertyGraphIndex(
    nodes=nodes,
    kg_extractors=[kg_extractor],
    property_graph_store=graph_store,
    show_progress=True,
)

Generating embeddings: 100%|██████████| 71/71 [01:01<00:00,  1.15it/s]


In [10]:
index.property_graph_store.get_triplets()[10]

[EntityNode(label='人物', embedding=None, properties={'creation_date': '2025-01-16', 'id': '孔明', 'last_modified_date': '2025-01-16', 'entity_description': '孔明，即諸葛亮，是蜀國的丞相，智謀過人，指揮南安之戰並成功俘虜夏侯楙。"$$$$"', 'file_size': 14777, 'file_path': 'data/92-Romance-of-the-Three-Kingdoms.txt', 'file_name': '92-Romance-of-the-Three-Kingdoms.txt', 'triplet_source_id': '9098c4c4-b91e-405d-9267-bf9454eb6b1b', 'file_type': 'text/plain'}, name='孔明'),
 Relation(label='使用', source_id='孔明', target_id='紅油櫃車', properties={'creation_date': '2025-01-16', 'last_modified_date': '2025-01-16', 'file_size': 20175, 'file_path': 'data/90-Romance-of-the-Three-Kingdoms.txt', 'file_name': '90-Romance-of-the-Three-Kingdoms.txt', 'relationship_description': '孔明命令左右取了十輛紅油櫃車到帳下，顯示他在軍事行動中使用這些車輛。"$$$$"', 'triplet_source_id': 'a06922b1-0519-4632-865b-48356702668f', 'file_type': 'text/plain'}),
 EntityNode(label='物品', embedding=None, properties={'creation_date': '2025-01-16', 'id': '紅油櫃車', 'last_modified_date': '2025-01-16', 'entity_d

In [11]:
index.property_graph_store.get_triplets()[10][1].properties

{'creation_date': '2025-01-16',
 'last_modified_date': '2025-01-16',
 'file_size': 20175,
 'file_path': 'data/90-Romance-of-the-Three-Kingdoms.txt',
 'file_name': '90-Romance-of-the-Three-Kingdoms.txt',
 'relationship_description': '孔明命令左右取了十輛紅油櫃車到帳下，顯示他在軍事行動中使用這些車輛。"$$$$"',
 'triplet_source_id': 'a06922b1-0519-4632-865b-48356702668f',
 'file_type': 'text/plain'}

### Build communities

In [12]:
index.property_graph_store.build_communities()

BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}

### Create QueryEngine

In [13]:
query_engine = GraphRAGQueryEngine(
    graph_store=index.property_graph_store,
    llm=gpt4o,
    index=index,
    similarity_top_k=10,
)

### Querying

In [ ]:
response = query_engine.query(
    "孔明和孟獲之間發生過什麼事？"
)
display(Markdown(f"{response.response}"))

In [ ]:
response = query_engine.query("孔明和孟獲之間發生過什麼事？")
display(Markdown(f"{response.response}"))

# Evaluation

## Testset

### RAGAS 自動生成目前有問題

https://github.com/explodinggradients/ragas/issues/1730

In [16]:
# from ragas.testset import TestsetGenerator

# generator = TestsetGenerator.from_llama_index(
#     llm=gpt4o,
#     embedding_model=embed_model,
# )

In [17]:
# # generate testset
# testset = generator.generate_with_llamaindex_docs(
#     documents,
#     testset_size=5,
# )

### No-LLM Evaluation Testset

In [ ]:
from ragas import EvaluationDataset

sample_queries = [
    "誰是祝融夫人？",
    "孟獲被諸葛亮第幾次擒住後才真心歸降？",
    "孔明用什麼方法平定了烏戈國的藤甲軍？",
    "趙雲在鳳鳴山與韓德及其四子交戰的結果如何？",
    "孔明如何智取南安城？",
]

expected_responses = [
    "祝融夫人是孟獲的妻子，南蠻祝融氏之後，善使飛刀，百發百中。",
    "孟獲被諸葛亮第七次擒住後才真心歸降，並誓不再反。",
    "孔明利用火攻之計，在盤蛇谷用火藥和火砲燒毀了烏戈國的藤甲軍。",
    "趙雲在鳳鳴山與韓德及其四子交戰，最終斬殺了韓德及其三子，並生擒了次子韓瑤。",
    "孔明利用崔諒和楊陵的內應計策，讓關興和張苞扮作安定軍馬進入南安城，最終擒住了夏侯楙。",
]

dataset = []

# for query, reference in zip(sample_queries, expected_responses):
#     relevant_docs = vector_retriever.invoke(query)
#     response = KG_chain.invoke(input=query)
#     dataset.append(
#         {
#             "user_input": query,
#             "retrieved_contexts": [rdoc.page_content for rdoc in relevant_docs],
#             "response": response,
#             "reference": reference,
#         }
#     )

# evaluation_dataset = EvaluationDataset.from_list(dataset)

In [ ]:
# from ragas import evaluate
# from ragas.llms import LangchainLLMWrapper
# from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

# evaluator_llm = LangchainLLMWrapper(gpt4o)

# result = evaluate(
#     dataset=evaluation_dataset,
#     metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],
#     llm=evaluator_llm,
# )

# result

## Evaluating the `QueryEngine` 

In [6]:
# # import metrics
# from ragas.metrics import (
#     Faithfulness,
#     AnswerRelevancy,
#     ContextPrecision,
#     ContextRecall,
# )

# # init metrics with evaluator LLM
# from ragas.llms import LlamaIndexLLMWrapper

# evaluator_llm = LlamaIndexLLMWrapper(gpt4o)
# metrics = [
#     Faithfulness(llm=evaluator_llm),
#     AnswerRelevancy(llm=evaluator_llm),
#     ContextPrecision(llm=evaluator_llm),
#     ContextRecall(llm=evaluator_llm),
# ]

In [ ]:
# # convert to Ragas Evaluation Dataset
# ragas_dataset = testset.to_evaluation_dataset()
# ragas_dataset

In [ ]:
# from ragas.integrations.llama_index import evaluate

# result = evaluate(
#     query_engine=query_engine,
#     metrics=metrics,
#     dataset=ragas_dataset,
# )

In [ ]:
# # final scores
# print(result)

In [ ]:
# result.to_pandas()